## TLDR I really like comprehensions, maps, and functional style programming, It may look weird at first until i explain it, then it makes a ton of sense

In [4]:
import pandas as pd
from sklearn.datasets import load_iris
data = load_iris()
temp = {}
temp['data'] = data['data'].tolist()
temp['target'] = data['target'].tolist()
df = pd.DataFrame(temp)

In [5]:
df

,data,target
0,"[5.1, 3.5, 1.4, 0.2]",0
1,"[4.9, 3.0, 1.4, 0.2]",0
2,"[4.7, 3.2, 1.3, 0.2]",0
3,"[4.6, 3.1, 1.5, 0.2]",0
4,"[5.0, 3.6, 1.4, 0.2]",0
...,...,...
145,"[6.7, 3.0, 5.2, 2.3]",2
146,"[6.3, 2.5, 5.0, 1.9]",2
147,"[6.5, 3.0, 5.2, 2.0]",2
148,"[6.2, 3.4, 5.4, 2.3]",2


In [6]:
stratified_data = {
    target:[
        vector 
        for vector, targ in zip(
            data['data'].tolist(),
            data['target'].tolist()) 
        if targ == target] 
    for target in list(set(data['target'].tolist()))
}

In [7]:
from statistics import stdev, mean
from math import pi, exp, sqrt
# yes this _does_ exist, and i dont want to implement std dev on my own.

In [8]:
def generate_probs(stratified_dict: dict) -> dict:
    def reshape_list(vector_list: list) -> list:
        return [[vec[i] for vec in vector_list] for i in range(len(vector_list[0]))]
    """
        alright this requires some explanation
        it creates a dict in such that it contains lambda functions, which serve as
        precompiled compute functions for the whole program
        thus you can select the class you are testing it against for classification
        then can use a functools reduce function on a map function to compute the classification comparison
        
        to do this though it requres the dict of list of vectors to be reshaped such that its column major
        rather than row major.
    """
    return {
        classification: [
            (lambda test: 1 / (sqrt(2 * pi * stdev(vals) ** 2)) * 
                          exp( 
                              -(test - mean(vals)) ** 2 / 
                              (2 * stdev(vals) ** 2)
                          ))
            for vals in reshape_list(stratified_dict[classification])] 
        for classification in stratified_dict.keys()
    }
        

In [9]:
""" now we have this function working we can create a function that compiles those lambdas
into a larger lambda that makes test functions for the classification"""
""" it does this via iteration of the vector tests and compute the product of the result"""

from math import prod
def compile_probs(probs: dict) -> dict:
    return {classification: (lambda test: 
                prod(map(lambda a: a[0](a[1]) , zip(probs[classification],test))))
            for classification in probs.keys()}
            

In [10]:
from functools import reduce
"""this then takes those classification probability functions and 
returns a function that calculates the classification with the highest proabbility
"""
def bayes_compiler(X: list, Y: list):
    return (lambda test: reduce(
                lambda left, right: left if (left[1] > right[1]) else right,
                map(lambda classifier: (classifier, 
                    compile_probs(
                        generate_probs({
                            y: [x for x, y_test in zip(X,Y) if y_test == y]
                            for y in Y
                        })
                    )[classifier](test)), Y))[0])
                

In [11]:
model = bayes_compiler(data['data'].tolist(), data['target'].tolist())

In [12]:
model([5.0,3.1,2.4,1.6])

2

In [13]:
print(*map(lambda a: f"{a} = class {model(a)}", [[5.0,3.1,2.4,1.6], [5.0, 3.1, 2.1, 0.6],[6.0, 3.1, 5.5, 2.0]]),sep="\n")

[5.0, 3.1, 2.4, 1.6] = class 2
[5.0, 3.1, 2.1, 0.6] = class 2
[6.0, 3.1, 5.5, 2.0] = class 2
